<a href="https://colab.research.google.com/github/juancuevas-ops/analiosis-molecula-del-cancer/blob/main/detrminar_funcion_microsatelite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from Bio import SeqIO
import csv

def analizar_arn_brca2(ruta_archivo, ruta_salida):
    """
    Analiza las secuencias de ARN en brca2.fasta y guarda las posibles funciones en un archivo CSV.

    Args:
        ruta_archivo (str): Ruta al archivo FASTA con las secuencias de ARN.
        ruta_salida (str): Ruta para guardar el archivo CSV con los resultados.
    """

    try:
        registros = list(SeqIO.parse(ruta_archivo, "fasta"))

        if not registros:
            print("El archivo FASTA está vacío.")
            return

        resultados_totales = []

        for registro in registros:
            secuencia = str(registro.seq).upper()

            resultados = {
                "identificador": registro.id,
                "posible_funcion": determinar_posible_funcion(secuencia)
            }
            resultados_totales.append(resultados)

        guardar_resultados_csv(resultados_totales, ruta_salida)
        print(f"Resultados guardados en {ruta_salida}.")

    except FileNotFoundError:
        print("Archivo no encontrado.")

def determinar_posible_funcion(secuencia):
    """Determina la posible función de la secuencia."""
    if len(secuencia) > 1000:
        return "Posible región codificante o reguladora compleja."
    elif len(detectar_repeticiones(secuencia)) > 10:
        return "Presencia de repeticiones sugiere posible función reguladora o estructural."
    else:
        return "Función indeterminada. Se requiere análisis adicional."

def detectar_repeticiones(secuencia, longitud_minima=5):
    """Detecta patrones de repetición en la secuencia."""
    repeticiones = {}
    for i in range(len(secuencia) - longitud_minima + 1):
        subseq = secuencia[i:i + longitud_minima]
        for j in range(i + longitud_minima, len(secuencia) - longitud_minima + 1):
            if secuencia[j:j + longitud_minima] == subseq:
                if subseq in repeticiones:
                    repeticiones[subseq] += 1
                else:
                    repeticiones[subseq] = 1
    return repeticiones

def guardar_resultados_csv(resultados_totales, ruta_salida):
    """Guarda los resultados en un archivo CSV."""
    with open(ruta_salida, "w", newline="") as archivo_csv:
        escritor = csv.writer(archivo_csv)
        encabezados = ["identificador", "posible_funcion"]
        escritor.writerow(encabezados)

        for resultados in resultados_totales:
            fila = [resultados["identificador"], resultados["posible_funcion"]]
            escritor.writerow(fila)

# Uso del script
ruta_arn_brca2 = "/content/rna.fna"  # Reemplaza con la ruta correcta
ruta_salida_csv = "resultados_brca2_funciones.csv" # Nombre adecuado para el CSV.

analizar_arn_brca2(ruta_arn_brca2, ruta_salida_csv)